In [1]:
# %%
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV

import matplotlib.pyplot as plt
#import seaborn as sns
import pickle



# %%
YEARS = [2018, 2019, 2020, 2021, 2022, 2023,2024]

# %%
data_all = pd.DataFrame()

def calculate_seconds(row):
    if row['qtr'] != 5:
        return 3600 - row['game_seconds_remaining']
    else:
        return 600 - row['game_seconds_remaining'] + 3600


def get_quarter_value(dataf):
    if 'END QUARTER' in dataf['desc']:
        return dataf['level_0']
    else:
        return None

for i in YEARS:  
    i_data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(i) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data_all = pd.concat([data_all,i_data])

ppr = 1

data = data_all.loc[data_all.season_type=='REG']
#data = data_all.loc[(data_all.play_type.isin(['no_play','pass','run'])) & (data_all.epa.isna()==False)]
#data.loc[data['pass']==1, 'play_type'] = 'pass'
#data.loc[data.rush==1, 'play_type'] = 'run'
data.reset_index(drop=True, inplace=True)
data['turnover'] = data['interception'] + data['fumble_lost']
data = data.dropna(subset=['posteam'])
data['inside_10'] = (data['yardline_100'] < 10).astype(int)
data['20+_play'] = (data['yards_gained'] > 19).astype(int)
data['short_pass'] = (data['air_yards'] < 10).astype(int)
data['medium_pass'] = ((data['air_yards'] > 9)&(data['air_yards']<20)).astype(int)
data['deep_pass'] = (data['air_yards'] > 19).astype(int)
data['end_zone_target'] = (data['yardline_100'] - data['air_yards']) <= 0
data['fantasy_points'] = (
    data['complete_pass'] * ppr +          # 1 point per completion
    data['touchdown'] * 6 +           # 6 points per touchdown
    data['yards_gained'] * 0.1        # 0.1 points per yard gained
)
data['distance_to_EZ_after_target'] = data['yardline_100'] - data['air_yards']


C:\Users\src15\AppData\Local\Temp\ipykernel_20064\2018476112.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['turnover'] = data['interception'] + data['fumble_lost']
C:\Users\src15\AppData\Local\Temp\ipykernel_20064\2018476112.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['turnover'] = data['interception'] + data['fumble_lost']
C:\Users\src15\AppData\Local\Temp\ipykernel_20064\2018476112.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inser

In [2]:
def total_finder(home_or_away,home_total,away_total):
    if home_or_away == 'home':
        total = home_total
    else:
        total = away_total 
    return total

In [3]:
    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]


    # derive implied team total from betting market data
    data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
    data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)

    # Use list comprehension with zip for more efficient row-wise operations
    data['implied_posteam_total'] = [
    total_finder(has_ball, home_number, away_number)
        for has_ball, home_number, away_number in zip(data['posteam_type'], data['home_implied_total'], data['away_implied_total'])
    ]

C:\Users\src15\AppData\Local\Temp\ipykernel_20064\2215997381.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
C:\Users\src15\AppData\Local\Temp\ipykernel_20064\2215997381.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)
C:\Users\src15\AppData\Local\Temp\ipykernel_20064\2215997381.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the r

In [4]:
    runs = data[data['rush']==1]
    
    df = runs[['rusher_player_name','rusher_player_id','posteam','rush','run_location','game_id','inside_10','yardline_100','ydstogo','yards_gained','fantasy_points','rush_touchdown','down','week','season','home_implied_total','away_implied_total','posteam_type']]


# Season-over-season Comparison

In [5]:
def load_models():
    with open('Models/rush_yardage_model.pkl', 'rb') as file:
        yardage_model = pickle.load(file)
    
    with open('Models/rushing_touchdown_model.pkl', 'rb') as file:
        touchdown_model = pickle.load(file)

    return yardage_model, touchdown_model

In [6]:
yardage_model, touchdown_model = load_models()

In [7]:
df['season'].value_counts()

season
2022    13755
2024    13585
2021    13512
2023    13416
2020    12778
2019    12574
2018    12449
Name: count, dtype: int64

In [8]:
predictors = [
    'yardline_100', 'ydstogo',
    'down','run_location']

test_df = df[predictors]

test_df = pd.get_dummies(test_df, columns=['run_location'], drop_first=True)


In [9]:
# Add predictions to the new dataset (optional)
df['xYards'] = yardage_model.predict(test_df)
df['xTDs'] = touchdown_model.predict_proba(test_df)[:, 1]
#df['xFPs'] = best_model.predict(test_df)
df['compositeXFP'] = df['xTDs'] * 6 + df['xYards'] * 0.1

C:\Users\src15\AppData\Local\Temp\ipykernel_20064\2012736952.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xYards'] = yardage_model.predict(test_df)
C:\Users\src15\AppData\Local\Temp\ipykernel_20064\2012736952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xTDs'] = touchdown_model.predict_proba(test_df)[:, 1]
C:\Users\src15\AppData\Local\Temp\ipykernel_20064\2012736952.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [10]:
rushers = df.groupby(['rusher_player_id','posteam','season']).agg({'rusher_player_name':'max','rush':'sum','xTDs':'sum','rush_touchdown':'sum','xYards':'sum','yards_gained':'sum','fantasy_points':'sum','compositeXFP':'sum'}).sort_values('compositeXFP',ascending=False)

In [11]:
rushers.sort_values('yards_gained',ascending=False)

,,,rusher_player_name,rush,xTDs,rush_touchdown,xYards,yards_gained,fantasy_points,compositeXFP
rusher_player_id,posteam,season,,,,,,,,
00-0032764,TEN,2020,D.Henry,378,13.078955,17.0,1608.787109,2027.0,304.7,239.352448
00-0034844,PHI,2024,S.Barkley,345,10.159079,13.0,1486.042603,2005.0,284.5,209.558731
00-0032764,BAL,2024,D.Henry,325,12.009431,16.0,1366.971680,1921.0,294.1,208.753769
00-0036223,IND,2021,J.Taylor,332,16.310417,18.0,1338.681274,1811.0,289.1,231.730621
00-0035700,LV,2022,J.Jacobs,340,8.877774,12.0,1460.878296,1653.0,237.3,199.354477
...,...,...,...,...,...,...,...,...,...,...
00-0035535,NYG,2021,D.Slayton,1,0.004200,0.0,4.755414,-13.0,-1.3,0.500741
00-0031763,LAC,2022,D.Carter,2,0.014678,0.0,9.287204,-15.0,-1.5,1.016787
00-0034160,SEA,2022,M.Dickson,2,0.005562,0.0,9.472637,-18.0,4.2,0.980636


In [12]:
rushers['rush'].describe()

count    2422.000000
mean       37.069777
std        64.724725
min         1.000000
25%         2.000000
50%         7.000000
75%        36.000000
max       378.000000
Name: rush, dtype: float64

In [13]:
rushers_qual = rushers[rushers['rush']>=49]

In [14]:
rushers_qual.reset_index().sort_values(['rusher_player_id', 'season'])

,rusher_player_id,posteam,season,rusher_player_name,rush,xTDs,rush_touchdown,xYards,yards_gained,fantasy_points,compositeXFP
249,00-0023500,MIA,2018,F.Gore,156,1.852786,0.0,689.140808,722.0,72.2,80.030800
157,00-0023500,BUF,2019,F.Gore,166,7.329346,2.0,690.450745,599.0,71.9,113.021156
175,00-0023500,NYJ,2020,F.Gore,187,4.254788,2.0,812.161194,653.0,77.3,106.744850
79,00-0025394,WAS,2018,A.Peterson,251,5.531588,7.0,1092.741211,1042.0,146.2,142.463654
127,00-0025394,WAS,2019,A.Peterson,211,5.189333,5.0,919.063477,898.0,119.8,123.042343
...,...,...,...,...,...,...,...,...,...,...,...
388,00-0039794,NYJ,2024,B.Allen,92,1.838184,2.0,386.411255,334.0,45.4,49.670231
484,00-0039874,MIA,2024,J.Wright,68,0.560679,0.0,305.062256,249.0,30.9,33.870300
315,00-0039875,BUF,2024,R.Davis,113,2.412854,3.0,491.143890,442.0,62.2,63.591515
382,00-0039910,WAS,2024,J.Daniels,65,4.203051,5.0,253.253510,325.0,62.5,50.543655


In [15]:
df = rushers_qual.reset_index().sort_values(['rusher_player_id', 'season'])

# Create lagged columns for the next season's stats
for col in ['xTDs', 'rush_touchdown', 'xYards', 'yards_gained','fantasy_points', 'compositeXFP']:
    df[f'{col}_next'] = df.groupby('rusher_player_id')[col].shift(-1)

# Drop rows where the next season's data is missing (last season for each player)
df_lagged = df.dropna(subset=[f'{col}_next' for col in ['xTDs', 'rush_touchdown', 'xYards', 'yards_gained','fantasy_points', 'compositeXFP']])

In [16]:
df_lagged

,rusher_player_id,posteam,season,rusher_player_name,rush,xTDs,rush_touchdown,xYards,yards_gained,fantasy_points,compositeXFP,xTDs_next,rush_touchdown_next,xYards_next,yards_gained_next,fantasy_points_next,compositeXFP_next
249,00-0023500,MIA,2018,F.Gore,156,1.852786,0.0,689.140808,722.0,72.2,80.030800,7.329346,2.0,690.450745,599.0,71.9,113.021156
157,00-0023500,BUF,2019,F.Gore,166,7.329346,2.0,690.450745,599.0,71.9,113.021156,4.254788,2.0,812.161194,653.0,77.3,106.744850
79,00-0025394,WAS,2018,A.Peterson,251,5.531588,7.0,1092.741211,1042.0,146.2,142.463654,5.189333,5.0,919.063477,898.0,119.8,123.042343
127,00-0025394,WAS,2019,A.Peterson,211,5.189333,5.0,919.063477,898.0,119.8,123.042343,7.808036,7.0,633.967468,604.0,102.4,110.244965
213,00-0027029,BUF,2018,L.McCoy,161,3.405171,3.0,707.724304,514.0,69.4,91.203461,4.445641,4.0,420.439240,465.0,70.5,68.717773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,00-0039021,CHI,2023,R.Johnson,81,2.721753,2.0,348.866028,352.0,47.2,51.217121,4.623554,6.0,210.167725,150.0,51.0,48.758095
359,00-0039032,TEN,2023,T.Spears,100,1.696942,2.0,439.213715,453.0,57.3,54.103027,2.697551,4.0,360.515991,312.0,55.2,52.236904
321,00-0039040,MIA,2023,D.Achane,103,3.115249,8.0,440.272461,800.0,128.0,62.718739,8.383015,6.0,845.044250,907.0,126.7,134.802521
139,00-0039139,DET,2023,J.Gibbs,182,6.807074,10.0,765.711670,945.0,154.5,117.413605,10.387108,16.0,1046.394409,1412.0,237.2,166.962082


In [17]:
df_lagged.tail(11)

,rusher_player_id,posteam,season,rusher_player_name,rush,xTDs,rush_touchdown,xYards,yards_gained,fantasy_points,compositeXFP,xTDs_next,rush_touchdown_next,xYards_next,yards_gained_next,fantasy_points_next,compositeXFP_next
145,00-0038120,NYJ,2023,Bre.Hall,223,2.784668,5.0,989.536926,994.0,129.4,115.661697,5.355914,5.0,910.763062,876.0,117.6,123.211792
86,00-0038134,SEA,2022,K.Walker,228,7.276539,9.0,968.212036,1050.0,159.0,140.480438,8.784863,8.0,921.949341,905.0,138.5,144.904114
72,00-0038134,SEA,2023,K.Walker,219,8.784863,8.0,921.949341,905.0,138.5,144.904114,6.630961,7.0,649.952393,573.0,99.3,104.781006
141,00-0038542,ATL,2023,B.Robinson,214,3.705592,4.0,946.075317,976.0,121.6,116.841087,10.814352,14.0,1292.946533,1456.0,229.6,194.180756
494,00-0038555,JAX,2023,T.Bigsby,50,1.936299,2.0,207.928543,132.0,25.2,32.410648,6.889011,7.0,689.010559,766.0,118.6,110.235130
432,00-0038794,DEN,2023,J.McLaughlin,76,1.265803,1.0,342.754364,410.0,47.0,41.870255,2.843175,1.0,495.620972,496.0,55.6,66.621147
378,00-0039021,CHI,2023,R.Johnson,81,2.721753,2.0,348.866028,352.0,47.2,51.217121,4.623554,6.0,210.167725,150.0,51.0,48.758095
359,00-0039032,TEN,2023,T.Spears,100,1.696942,2.0,439.213715,453.0,57.3,54.103027,2.697551,4.0,360.515991,312.0,55.2,52.236904
321,00-0039040,MIA,2023,D.Achane,103,3.115249,8.0,440.272461,800.0,128.0,62.718739,8.383015,6.0,845.044250,907.0,126.7,134.802521
139,00-0039139,DET,2023,J.Gibbs,182,6.807074,10.0,765.711670,945.0,154.5,117.413605,10.387108,16.0,1046.394409,1412.0,237.2,166.962082


In [18]:
correlations = {}

# Loop through each stat to calculate its correlation with the following season
for col in ['xTDs', 'rush_touchdown', 'xYards', 'yards_gained','fantasy_points', 'compositeXFP']:
    correlation = df_lagged[col].corr(df_lagged[f'{col}_next'])
    #print(df_lagged)
    correlations[col] = correlation

# Display the results
correlations_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Correlation with Next Season'])
print(correlations_df)

                Correlation with Next Season
xTDs                                0.357824
rush_touchdown                      0.318376
xYards                              0.527077
yards_gained                        0.490667
fantasy_points                      0.446454
compositeXFP                        0.488423


In [19]:
#correlations_df.drop(['xFPs'],inplace=True)

In [20]:
correlations = {}

# Loop through each stat to calculate its correlation with the following season
for metric in ['rush_touchdown', 'yards_gained','fantasy_points']:
    if metric == 'rush_touchdown':
        predictor = 'xTDs'
    if metric == 'yards_gained':
        predictor = 'xYards'
    if metric == 'fantasy_points':
        predictor = 'compositeXFP'
    #print(f"{predictor} to predict {metric}")
    correlation = df_lagged[predictor].corr(df_lagged[f'{metric}_next'])
    correlations[metric] = correlation

# Display the results
x_stat_correlations_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Expected Stat Correlation with Actual Next Season Target'])
print(x_stat_correlations_df)

                Expected Stat Correlation with Actual Next Season Target
rush_touchdown                                           0.305842       
yards_gained                                             0.462009       
fantasy_points                                           0.422387       


In [21]:
correlations_df.merge(x_stat_correlations_df,right_index=True,left_index=True,how='outer').sort_values('Correlation with Next Season',ascending=False).dropna(axis=0)

,Correlation with Next Season,Expected Stat Correlation with Actual Next Season Target
yards_gained,0.490667,0.462009
fantasy_points,0.446454,0.422387
rush_touchdown,0.318376,0.305842


In [22]:
df_lagged[['xTDs','rush_touchdown','rush_touchdown_next','rush']].corr()

,xTDs,rush_touchdown,rush_touchdown_next,rush
xTDs,1.000000,0.828272,0.305842,0.797430
rush_touchdown,0.828272,1.000000,0.318376,0.644817
rush_touchdown_next,0.305842,0.318376,1.000000,0.330588
rush,0.797430,0.644817,0.330588,1.000000


In [23]:
df_lagged[['xYards','yards_gained','yards_gained_next','rush']].corr()

,xYards,yards_gained,yards_gained_next,rush
xYards,1.000000,0.951214,0.462009,0.998217
yards_gained,0.951214,1.000000,0.490667,0.953042
yards_gained_next,0.462009,0.490667,1.000000,0.460092
rush,0.998217,0.953042,0.460092,1.000000


# Intra-season Comparison

In [24]:
    runs = data[data['rush']==1]
    
    df = runs[['rusher_player_name','rusher_player_id','posteam','rush','run_location','game_id','inside_10','yardline_100','ydstogo','yards_gained','fantasy_points','rush_touchdown','down','week','season','home_implied_total','away_implied_total','posteam_type']]


In [25]:
predictors = [
    'yardline_100', 'ydstogo',
    'down','run_location']

test_df = df[predictors]

test_df = pd.get_dummies(test_df, columns=['run_location'], drop_first=True)


In [26]:


# Add predictions to the new dataset (optional)
df['xYards'] = yardage_model.predict(test_df)
df['xTDs'] = touchdown_model.predict_proba(test_df)[:,1]
#df['xFPs'] = best_model.predict(test_df)
df['compositeXFP'] = df['xTDs'] * 6 + df['xYards'] * 0.1

C:\Users\src15\AppData\Local\Temp\ipykernel_20064\3376893192.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xYards'] = yardage_model.predict(test_df)
C:\Users\src15\AppData\Local\Temp\ipykernel_20064\3376893192.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xTDs'] = touchdown_model.predict_proba(test_df)[:,1]
C:\Users\src15\AppData\Local\Temp\ipykernel_20064\3376893192.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [27]:

# Filter weeks 1-9 and weeks 10-18
first_half = df[df['week'].between(1, 9)]
second_half = df[df['week'].between(10, 18)]

# Define metrics to aggregate
metrics = ['rush','xTDs', 'rush_touchdown', 'xYards', 'yards_gained', 'fantasy_points', 'compositeXFP']

# Aggregate metrics for each player-season for weeks 1-9
first_half_agg = first_half.groupby(['rusher_player_id', 'season'])[metrics].sum()
first_half_agg = first_half_agg.add_suffix('_wk1_9')

# Aggregate metrics for each player-season for weeks 10-18
second_half_agg = second_half.groupby(['rusher_player_id', 'season'])[metrics].sum()
second_half_agg = second_half_agg.add_suffix('_wk10_18')

# Merge the two aggregated DataFrames
df_aggregated = pd.merge(first_half_agg, second_half_agg, left_index=True, right_index=True)


In [28]:
df_aggregated = df_aggregated[(df_aggregated['rush_wk1_9'] >= 20) & (df_aggregated['rush_wk10_18'] >= 20)]


In [29]:
# Create a dictionary to store correlations for each metric
correlations = {}

# Calculate correlation for each metric between weeks 1-9 and weeks 10-18
for metric in metrics:
    correlation = df_aggregated[f'{metric}_wk1_9'].corr(df_aggregated[f'{metric}_wk10_18'])
    correlations[metric] = correlation

# Display the correlation results
correlations_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Correlation between Weeks 1-9 and 10-18'])
print(correlations_df)


                Correlation between Weeks 1-9 and 10-18
rush                                           0.581604
xTDs                                           0.410237
rush_touchdown                                 0.388956
xYards                                         0.587901
yards_gained                                   0.574520
fantasy_points                                 0.538199
compositeXFP                                   0.546767


In [30]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','rush_wk1_9']].corr()

,fantasy_points_wk1_9,compositeXFP_wk1_9,fantasy_points_wk10_18,rush_wk1_9
fantasy_points_wk1_9,1.000000,0.903203,0.538199,0.877891
compositeXFP_wk1_9,0.903203,1.000000,0.500954,0.973474
fantasy_points_wk10_18,0.538199,0.500954,1.000000,0.511548
rush_wk1_9,0.877891,0.973474,0.511548,1.000000


In [32]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','rush_wk1_9','rush_wk10_18','yards_gained_wk1_9','xYards_wk1_9','yards_gained_wk10_18','xTDs_wk1_9','rush_touchdown_wk1_9','rush_touchdown_wk10_18']].corr()[['yards_gained_wk10_18']].sort_values('yards_gained_wk10_18',ascending=False)

,yards_gained_wk10_18
yards_gained_wk10_18,1.000000
rush_wk10_18,0.945646
fantasy_points_wk10_18,0.942424
rush_touchdown_wk10_18,0.669581
yards_gained_wk1_9,0.574520
xYards_wk1_9,0.536699
rush_wk1_9,0.533838
fantasy_points_wk1_9,0.531066
compositeXFP_wk1_9,0.508092
xTDs_wk1_9,0.375322


In [33]:
df_aggregated[['yards_gained_wk1_9','xYards_wk1_9','yards_gained_wk10_18','rush_wk1_9']].corr()

,yards_gained_wk1_9,xYards_wk1_9,yards_gained_wk10_18,rush_wk1_9
yards_gained_wk1_9,1.000000,0.937945,0.574520,0.938216
xYards_wk1_9,0.937945,1.000000,0.536699,0.997859
yards_gained_wk10_18,0.574520,0.536699,1.000000,0.533838
rush_wk1_9,0.938216,0.997859,0.533838,1.000000


In [34]:
df_aggregated[['rush_touchdown_wk1_9','xTDs_wk1_9','rush_touchdown_wk10_18','rush_wk1_9']].corr()

,rush_touchdown_wk1_9,xTDs_wk1_9,rush_touchdown_wk10_18,rush_wk1_9
rush_touchdown_wk1_9,1.000000,0.779615,0.388956,0.616439
xTDs_wk1_9,0.779615,1.000000,0.355004,0.767069
rush_touchdown_wk10_18,0.388956,0.355004,1.000000,0.373527
rush_wk1_9,0.616439,0.767069,0.373527,1.000000


In [35]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','rush_wk1_9']].corr()

,fantasy_points_wk1_9,compositeXFP_wk1_9,fantasy_points_wk10_18,rush_wk1_9
fantasy_points_wk1_9,1.000000,0.903203,0.538199,0.877891
compositeXFP_wk1_9,0.903203,1.000000,0.500954,0.973474
fantasy_points_wk10_18,0.538199,0.500954,1.000000,0.511548
rush_wk1_9,0.877891,0.973474,0.511548,1.000000
